In [1]:
import pandas as pd


In [2]:
moji_df = pd.read_csv('../data/raw/hate_speech_and_offensive_language_dataset.csv')
moji_df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [4]:
moji_df = moji_df.rename(columns={'tweet':'text'})

In [ ]:
# class 0 = hate speech, class 1 = offensive language
# class 2 = neither
moji_df['has_toxicity'] = moji_df['class'].apply(lambda x: 1 if x in [0,1] else 0)

In [6]:
moji_df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,text,has_toxicity
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1


In [9]:
import re

def clean_text(text):
    text = re.sub(r"http\S+", "", text)                  # удаление ссылок
    text = re.sub(r"@\w+", "", text)                     # упоминания @username
    text = re.sub(r"[^A-Za-z0-9\s.,!?\'\"-]", "", text)  # удаление спецсимволов
    text = re.sub(r"\s+", " ", text).strip()             # пробелы
    return text

moji_df['clean_text'] = moji_df['text'].apply(clean_text)
moji_df['clean_text'] = moji_df['clean_text'].str.lower()

In [17]:
moji_df.to_csv('../data/processed/moji_cleaned.csv')

In [ ]:
moji_df[['has_toxicity']].value_counts()
# 1 - toxic
# 0 - non_toxic

has_toxicity
1               20620
0                4163
Name: count, dtype: int64

In [15]:
non_toxic_df = moji_df[moji_df['has_toxicity'] == 0]
toxic_df = moji_df[moji_df['has_toxicity'] == 1].sample(n=len(non_toxic_df), random_state=42)

balanced_df = pd.concat([toxic_df, non_toxic_df]).sample(frac=1, random_state=42).reset_index(drop=True)

In [16]:
balanced_df['has_toxicity'].value_counts()

has_toxicity
0    4163
1    4163
Name: count, dtype: int64

In [18]:
balanced_df.to_csv('../data/processed/moji_balanced.csv')